# Set kernel to: `py-pythonEnv`
If this kernel is not available, see the Kernel Installation Instructions section located in at the bottom of this Notebook.

# Data Science on the C3 AI Suite
## C3DS - Python Methods \& APIs
#### For questions, comments, or concerns, please email the C3.ai Academy Team \< <training@c3.ai> \>, discuss with your Trainer, or leave your thoughts in the Google Forms at the end of this module.

## Table of Contents

__[C3DS Python Methods](#part1)__

* [Section 1 - Define \& Declare a New Python Method](#1)


* [Section 2 - Implement the Python Method](#2)


__[C3DS Python API](#part2)__


* [Section 3 - Testing Both Methods](#3) 
    * [A. Testing on the Platform](#3.A)
    * [B. Testing in this Jupyter Notebook](#3.B)


* [Section 4 - Calculate Statistics](#4)


* [Section 5 - Writing the R Implementation (Optional)](#5)
    * [A. Test the pearsonCorerelationR Function](#5.A)
    * [B. Visualize the Correlation using SmartBulb Data](#5.B)

[Appendix - Kernel Installation Instructions](#apdx)

# C3DS Python Methods <a class="anchor" id="part1">

## Exercise Goal    
In this example shown below, we will reimplement the `lifeSpanInYears` JavaScript method that was implemented earlier in the Coursera course using `Python`. In doing so, you will define and develop a new method on `SmartBulb` and make the appropreiate field declarations thereof. Once you have successfully completed this, you will verify your `python` implentation will return the same information as the pre-existing JavaScript method.
    

## Section 1 - Define \& Declare a New Python Method <a class="anchor" id="1"></a>
Declare a new method to the `SmartBulb` Type. Name this **new method** `lifeSpanInYearsPythonMethod`. In doing so, pass a string as an input (name it `bulbId`) and recieve a `double` as the returned value to the `lifeSpanInYearsPythonMethod`.

Add a field annotation, written as `@py(env='pythonEnv')`, to indicate that it is a `python` method. Include Python environment's name that you created in the previous section in the field annotation.

> Note: The `env` variable does not have **py-** from the environment name; this is implied and will fail if included.

The implementation should look as follows:
```js
/**
* Returns the life span of this smartBulb using Python implementation
*/
@py(env='pythonEnv')
lifeSpanInYearsPythonMethod: function(bulbId: string): double
```

## Section 2 - Implement the Python Method <a class="anchor" id="2"></a>

Instead of having you write the python code, lets walk through the **implementation logic** behind the python code that generates the correct life spans. In the cell below, we implement `python` code as follows:
* Get `startDate` from the `SmartBulb` Type using the **C3** connector;
* Get the `endDate`:
    * `Fetch` records from `SmartBulbMeasurement` based on the logic filter (obtain the `objs` variable) using C3 connector;
    * Use Pandas to drop version (for demo here) and sort the column named `end` and pick the top value;
* Convert both `startDate` and `endDate` to required format using Python's datetime library; and
* Return the difference between `endDate` \& `startDate` as type double.

>Note: Create a new file in your package `src/types/SmartBulb.py`, then copy-paste the following method implementation. Save the file and reprovision. 

In [ ]:
import pandas as pd
import datetime as dt

def ISO_8601_strdate_to_datetime(str_date):
    return dt.datetime.strptime(str_date[:-6], "%Y-%m-%dT%H:%M:%S")

def abs_day_diff(date_1_str, date_2_str):
    date_1 = ISO_8601_strdate_to_datetime(date_1_str)
    date_2 = ISO_8601_strdate_to_datetime(date_2_str)
    return abs((date_1 - date_2).days) / 365.

def lifeSpanInYearsPythonMethod(bulbId):
    #Get startDate from SmartBulb
    bulb = c3.SmartBulb.get(this={'id': bulbId})
    startDate = bulb.startDate

    #Get endDate from SmartBulbMeasurement
    limitSpec = -1
    defectFilter = "status == 1 && lumens == 0 && parent.id == '" + 'SBMS_serialNo_' + bulb.id + "'"
    smartbulbmeasurement_objs = c3.SmartBulbMeasurement.fetch(spec={'limit':limitSpec,
                                              'filter':defectFilter})["objs"]
    smartbulbmeasurement_pd = pd.DataFrame(smartbulbmeasurement_objs.toJson())
    smartbulbmeasurement_pd.drop(["version"], axis=1, inplace=True)
    smartbulbmeasurement_pd.sort_values(by='end',ascending=False)
    endDate = str(smartbulbmeasurement_pd[['end']].iloc[0].values[0])

    return abs_day_diff(startDate, endDate)

## Section 3 - Testing Both Methods  <a class="anchor" id="3"></a>


### A. Testing on the Platform <a class="anchor" id="3.A"></a>

After implementing the changes to your package (i.e., `SmartBulb.c3typ` and `SmartBulb.py`), save and provision the latest version of the code to your environment. Load appropriate tenant and tag and run the following two commands **in your console** and verify the result from the image given below:

```js
SmartBulb.lifeSpanInYears('SMBLB1')
SmartBulb.lifeSpanInYearsPythonMethod('SMBLB1')
```

**Expected Output**:
<img src="http://learnc3.ai/wp-content/uploads/2020/02/pythonMethodsResult.png" width="400"/>

> Note: Although these methods are similar, they are not exact. Can we understand why that may be?
> Note: To get this method to run in the notebook, you will need to refresh the notebook itself. Try refreshing the page, or **Close & Halt**, then reopen.

#### Please be patient, this may take up to a few minutes. 
> Note: If you are recieving an **HTTP Error 500** when running the python method in console, check you have data loaded by running `c3Grid(SmartBulbMeasurent.fetch())` in the console (or appropreiate syntax in Jupyter). If the console returns `no content`, you do not have data loaded and therefore will not execute properly. Refer to Data Loading Exercise in C3F or ask your instructor for guidance. If, after running the above command, you see a data tabel populated with information, no action is needed.

### B. Testing in this Jupyter Notebook <a class="anchor" id="3.B"></a>
Run the below cell to verify these results are what was returned to console. Go ahead and change the `calcLifeSpanOn` variable to see how the life span of different bulbs changes. Can you determine why this might be the case?

In [2]:
calcLifeSpanOn="SMBLB3"
jsYears = c3.SmartBulb.lifeSpanInYears(bulbId=calcLifeSpanOn)
pyYears = c3.SmartBulb.lifeSpanInYearsPythonMethod(bulbId=calcLifeSpanOn)

print("\tJS implementation for {0} calculated: {1:.5f} years".format(calcLifeSpanOn,jsYears))
print("\tPython implementation for {0} calculated: {1:.5f} years".format(calcLifeSpanOn,pyYears))

	JS implementation for SMBLB3 calculated: 3.71769 years
	Python implementation for SMBLB3 calculated: 3.71781 years


## Section 4 - Calculating Statistics <a class="anchor" id="4">
    
For the following exercise, use the Python API to calculate common percentile stats for a metrics. Here are the partial snippet of codes you will need, but please fill in the remaining steps, in the function you'll add to the `SmartBulb.py` file, with what you learnt from the above example!

Create a `SmartBulbStats` type, a similar process to what you did previously in this exercise, with the following field notation:

```js
type SmartBulbStats {
    // Minimum value of the timeseries
    min: double

    // Maximum value of the timeseries
    max: double

    // Average value of the timeseries
    avg: double

    // Standard deviation of the timeseries
    stddev: double

    // Median value of the timeseries
    median: double

    // First quartile of the timeseries
    p25: double

    // Third quartile of the timeseries
    p75: double

    // Average ratio of missing values in the timeseries (between 0 and 100)
    missingRatio: double
}
```

Add this function to `SmartBulb.py`, and replace the **???** with their appropreiate declarations:

> Note: Populate the `stats` dictionary object with information from the `data` NumPy array object (i.e., how would you find the `min()`, `max()`, etc., of a NumPy array?).

```py
import numpy as np

def computeMetricStats(spec):
    # evaluate the metric: evalMetric returns a Timeseries object
    ts = c3.SmartBulb.evalMetric(spec=spec)

    # we access the data of the Timeseries with the field m_data
    data = np.array(ts.m_data)

    # we compute the quartiles and the median in one shot
    p25, p50, p75 = np.percentile(data, [25, 50, 75])
    stats = {
        'min': ???,
        'max': ???,
        'avg': ???,
        'stddev': ???,
        'p25': ???,
        'median': ???,
        'p75': ???,
        'missingRatio': np.mean(ts.m_missing)  # for the missing ratio, we use the field m_missing
    }
    return stats
```

Finally, add the field declaration in the `SmartBulb.c3typ` file. The implentation should look as follows:
>Note: Were using the `training` field annotation here, not the `pythonEnv` annotation. 

```js
/**
 * Computes basic stats for a given Metric
 */
 @py(env='training')
 computeMetricStats: function(spec: EvalMetricSpec): SmartBulbStats  
```

In [1]:
mySpec = c3.EvalMetricSpec(
        id = "SMBLB1",
        expression = "AverageLumens",
        start = "2011-01-01",
        end = "2016-01-01",
        interval = "DAY"
)

In [2]:
jsYears = c3.SmartBulb.computeMetricStats(mySpec)

print(jsYears)

c3.SmartBulbStats(
 min=0.0,
 max=540.0958333333334,
 avg=240.36819322745527,
 stddev=154.83271833843457,
 median=278.9208333333333,
 p25=114.79375000000002,
 p75=371.9270833333333,
 missingRatio=18.025739320920042)


**Expected Output**:

```js
{u'avg': 3.6337268993839835,
 u'max': 8.625,
 u'median': 4.291666666666666,
 u'min': 0.0,
 u'missingRatio': 22.515400410677618,
 u'p25': 6.041666666666666,
 u'p75': 6.041666666666666,
 u'stddev': 2.562562291759538}
 ```
 
 

## Section 5 - Writing the R Implementation (Optional) <a class="anchor" id="5"></a>

In addition to JavaScript and Python, it is also possible to implement methods using R.
Inspect the `R` runtime in your `seed/ActionRuntimes/r-training.json` file:

```json
    {   
    "id": "r-training" ,
    "name": "r-training",
    "language": "R",
    "runtime": "Rscript"
    }
```

Declare a method to the `SmartBulb` Type. Name the new method as `pearsonCorrelationR`. Pass in two arrays of doubles, and recieve the correlation of the arrays as a double to the method.


Add a **field annotation** to indicate that it is a **R** method. Include the R environment's name that you created in the previous section in the field annotation.

> Note: The `env` variable does not have **r-** from the environment name; this is implied and will fail if included.

Now, create a file `SmartBulb.R` in the same folder where `SmartBulb.c3typ` resides. Include the following code for `pearsonCorrelationR` **method** into `SmartBulb.R`. The implementation should look as follows:
```r
pearsonCorrelationR <- function(doubleArray1, doubleArray2){

    res <- cor.test(doubleArray1, doubleArray2,
                        method = "pearson")

    corr <- res$estimate[[1]]

    return(corr)
} ```

... and now save and re-provision your code!!

### A. Test the `pearsonCorerelationR` Function <a class="anchor" id="5.A"></a>  

In [ ]:
c3.SmartBulb.pearsonCorrelationR([1,2,3],[1,10,100])

**Expected Output**:

```json
0.904194430179465
 ```

### B. Visualize the Correlation using SmartBulb Data<a class="anchor" id="5.B"></a>  

In [ ]:
start = datetime(2011,1,1)
end = datetime(2016,1,1)
interval = 'DAY'
expressions = ["AverageLumens","AveragePower","AverageVoltage","AverageTemperature"]
spec = c3.EvalMetricsSpec(expressions = expressions,
                          ids = ['SMBLB1'],
                          interval = interval,
                          start = start,
                          end = end,
                          limit = 100)

res = c3.SmartBulb.evalMetrics(spec=spec)

ds = c3.Dataset.fromEvalMetricsResult(result = res)
df = c3.Dataset.toPandas(dataset = ds)
df['source'] = df.index.str.split('_').str[0]
df['timestamp'] = pd.to_datetime(df.index.str.split('_').str[1],format="%Y-%m-%dT%H:%M:%S.%f")
df.set_index('timestamp',inplace=True)

In [ ]:
df.plot()
plt.show()

In [ ]:
c3.SmartBulb.pearsonCorrelationR(list(df.AveragePower.values),list(df.AverageVoltage.values))

**Expected Output**:
```json
0.99901320315066
 ```

In [ ]:
c3.SmartBulb.pearsonCorrelationR(list(df.AverageLumens.values),list(df.AveragePower.values))

**Expected Output**:

```json
0.996807868107811
 ```

In [ ]:
c3.SmartBulb.pearsonCorrelationR(list(df.AverageLumens.values),list(df.AverageTemperature.values))

**Expected Output**:
```json
0.672108252951516
 ```
 

## Conclusion
Congratulations! You have successfully reimplemented the `lifeSpanInYears` JavaScript methodin `Python`! In doing so, you **defined and developed** a new method on `SmartBulb` making the appropreiate field declarations thereof and verified your `python` implentation returned the same information as the pre-existing JavaScript method.

## Make sure to CLOSE AND HALT  (from bottom of the File menu at the top) this notebook when you leave.

## Appendix - Kernel Installation Instructions <a class="anchor" id="apdx">
If the **`py-pythonEnv`** kernel is not available, perform the following:
1. Select the 'Kernel' tab at the top of this notebook, then select 'Manage Kernels';
2. Scroll to find `py-pythonEnv`, and click the 'Install' button;
> Note: This installation will "fail", but don't worry, exit the Manage Kernels UI and return to the main notebook.
3. Navigate to the 'File' tab located at the top of the notebook, and select 'Close and Hault'. This will close the notebook. Re-open it.
4. You should notice, in the upper-right corner, the kernel is now set correctly to `py-pythonEnv`. If it is not, navigate to the Kernel tab, then select 'Change Kernel' and select the appropriate option.